In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [25]:
cases = spark.read.csv('case.csv', sep=",", header=True, inferSchema=True)

In [3]:
dept = spark.read.csv('dept.csv', sep=",", header=True, inferSchema=True)

In [4]:
source = spark.read.csv('source.csv', sep=",", header=True, inferSchema=True)

In [81]:
cases.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 SLA_due_date         | 2020-09-26 00:42:00                  
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 005                                  
 days_past_SLA        | 233                                  
 day_to_

In [6]:
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [7]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [26]:
fmt = 'M/d/yy H:mm'

cases = (
    cases.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))
    .withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))
    .withColumn('SLA_due_date', to_timestamp('SLA_due_date', fmt))
)

In [27]:
cases.show(1,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [36]:
cases_opened = (
    cases.filter(expr('case_closed == "NO"'))
    .withColumn('days_past_SLA', datediff(current_timestamp(), 'SLA_due_Date'))
    .withColumn('day_to_close', datediff('case_closed_date', 'case_opened_date'))
)

In [79]:
cases_opened.select(max('days_past_SLA')).show()

+------------------+
|max(days_past_SLA)|
+------------------+
|              1581|
+------------------+



## 2. How many Stray Animal cases are there?


In [40]:
cases.filter(expr('service_request_type == "Stray Animal"')).count()

26760

## 3.How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [49]:
(
    cases.filter(expr('dept_division == "Field Operations"'))
    .filter(expr('service_request_type != "Officer Standby"'))
).count()

113902

## 4. Convert the council_district column to a string column.



In [51]:
cases = cases.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [55]:
cases.select('council_district').show

<bound method DataFrame.show of DataFrame[council_district: string]>

## 5. Extract the year from the case_closed_date column.


In [57]:
from pyspark.sql.functions import month, year, quarter


In [64]:
cases = cases.withColumn('year_closed', year('case_closed_date'))

In [65]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 days_past_SLA        | 233                  
 day_to_close         | 0                    
 year_closed          | 2018                 
only showing top 1 row



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [70]:
cases = cases.withColumn('num_hours_late', expr('num_days_late * 24'))

In [71]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 days_past_SLA        | 233                  
 day_to_close         | 0                    
 year_closed          | 2018                 
 num_hours_late       | -23964.2102784       
only showing top 1 row



## 7. Join the case data with the source and department data.



In [80]:
source.show(1, vertical=True), dept.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



(None, None)

In [92]:
cases = cases.join(source, on=cases.source_id == source.source_id, how="left").drop(source.source_id)

In [94]:
cases = cases.join(dept, on=cases.dept_division == dept.dept_division, how="left").drop(dept.dept_division)

In [95]:
cases.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 dept_division          | Field Operations                     
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 source_id              | svcCRMLS                             
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 005                                  
 days_past_SLA          | 233           

## 8. Are there any cases that do not have a request source?

